<a href="https://colab.research.google.com/github/ishaanpaul98/Sentiment-Analysis/blob/main/Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports #

In [246]:
# Import libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from wordcloud import WordCloud
import re

from bs4 import BeautifulSoup
from urllib.request import urlopen, Request
from nltk.sentiment.vader import SentimentIntensityAnalyzer

import datetime
import yfinance as yf
import time
import requests
import io

In [247]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [248]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

In [249]:
from sklearn.metrics import accuracy_score,precision_score,recall_score,confusion_matrix,roc_curve,classification_report
from sklearn.metrics import plot_confusion_matrix

# Stock Data Helper Functions #

In [250]:
def getStockDataDaily(symbol, day):
    print("Getting stock data for stock $"+symbol)
    df = yf.download(symbol, start=day, period = "1d")
    return df

getStockDataDaily('AAPL', "2022-12-21")

Getting stock data for stock $AAPL
[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-12-21,132.979996,136.809998,132.750000,135.449997,135.449997,85928000
2022-12-22,134.350006,134.559998,130.300003,132.229996,132.229996,77852100
2022-12-23,130.919998,132.419998,129.639999,131.860001,131.860001,63814900
2022-12-27,131.380005,131.410004,128.720001,130.029999,130.029999,69007800
2022-12-28,129.669998,131.029999,125.870003,126.040001,126.040001,85438400
2022-12-29,127.989998,130.479996,127.730003,129.610001,129.610001,75703700
2022-12-30,128.410004,129.949997,127.430000,129.929993,129.929993,76960600
2023-01-03,130.279999,130.899994,124.169998,125.070000,125.070000,112117500
2023-01-04,126.889999,128.660004,125.080002,126.360001,126.360001,89113600


In [251]:
def arrayToString(arr):
    print("Starting array to list")
    listToStr = ' '.join([str(elem) for elem in arr])
    return listToStr


In [252]:
def getMultiStockDataDaily(symbols, day):
    if len(symbols) > 1:
        print("Length of symbols array is more than 1. STARTING ARRAYTOSTRING")
        symbols = arrayToString(symbols)
    print("Getting stock data for stock $"+symbols)
    df = yf.download(symbols, start=day, period = "1d", group_by='ticker')
    return df

getMultiStockDataDaily(['AAPL', 'TSLA'], "2022-12-21")

Length of symbols array is more than 1. STARTING ARRAYTOSTRING
Starting array to list
Getting stock data for stock $AAPL TSLA
[*********************100%***********************]  2 of 2 completed


TSLA                                                  \
                  Open        High         Low       Close   Adj Close   
Date                                                                     
2022-12-21  139.339996  141.259995  135.889999  137.570007  137.570007   
2022-12-22  136.000000  136.630005  122.260002  125.349998  125.349998   
2022-12-23  126.370003  128.619995  121.019997  123.150002  123.150002   
2022-12-27  117.500000  119.669998  108.760002  109.099998  109.099998   
2022-12-28  110.349998  116.269997  108.239998  112.709999  112.709999   
2022-12-29  120.389999  123.570000  117.500000  121.820000  121.820000   
2022-12-30  119.949997  124.480003  119.750000  123.180000  123.180000   
2023-01-03  118.470001  118.800003  104.639999  108.099998  108.099998   
2023-01-04  109.110001  114.589996  107.519997  113.639999  113.639999   
2023-01-05  110.510002  111.750000  107.160004  110.339996  110.339996   
2023-01-06  103.000000  114.389999  101.809998  113.059998  113.059998   
2023-01-09  118.959999  123.519997  117.110001  119.769997  119.769997   
2023-01-10  121.070000  122.760002  114.919998  118.849998  118.849998   
2023-01-11  122.089996  125.949997  120.510002  123.220001  123.220001   
2023-01-12  122.559998  124.129997  117.000000  123.559998  123.559998   
2023-01-13  116.550003  122.629997  115.599998  122.400002  122.400002   
2023-01-17  125.699997  131.699997  125.019997  131.490005  131.490005   
2023-01-18  136.559998  136.679993  127.010002  128.779999  128.779999   
2023-01-19  127.260002  129.990005  124.309998  127.169998  127.169998   
2023-01-20         NaN         NaN         NaN         NaN         NaN   

                               AAPL                                      \
                 Volume        Open        High         Low       Close   
Date                                                                      
2022-12-21  145417400.0  132.979996  136.809998  132.750000  135.449997   
2022-12-22  210090300.0  134.350006  134.559998  130.300003  132.229996   
2022-12-23  166989700.0  130.919998  132.419998  129.639999  131.860001   
2022-12-27  208643400.0  131.380005  131.410004  128.720001  130.029999   
2022-12-28  221070500.0  129.669998  131.029999  125.870003  126.040001   
2022-12-29  221923300.0  127.989998  130.479996  127.730003  129.610001   
2022-12-30  157304500.0  128.410004  129.949997  127.430000  129.929993   
2023-01-03  231402800.0  130.279999  130.899994  124.169998  125.070000   
2023-01-04  180389000.0  126.889999  128.660004  125.080002  126.360001   
2023-01-05  157986300.0  127.129997  127.769997  124.760002  125.019997   
2023-01-06  220575900.0  126.010002  130.289993  124.889999  129.619995   
2023-01-09  190284000.0  130.470001  133.410004  129.889999  130.149994   
2023-01-10  167642500.0  130.259995  131.259995  128.119995  130.729996   
2023-01-11  183810800.0  131.250000  133.509995  130.460007  133.490005   
2023-01-12  169400900.0  133.880005  134.259995  131.440002  133.410004   
2023-01-13  180439300.0  132.029999  134.919998  131.660004  134.759995   
2023-01-17  186477000.0  134.830002  137.289993  134.130005  135.940002   
2023-01-18  195680300.0  136.820007  138.610001  135.029999  135.210007   
2023-01-19  170291900.0  134.080002  136.250000  133.770004  135.270004   
2023-01-20          NaN  135.279999  136.889999  134.220001  136.804993   

                                   
             Adj Close     Volume  
Date                               
2022-12-21  135.449997   85928000  
2022-12-22  132.229996   77852100  
2022-12-23  131.860001   63814900  
2022-12-27  130.029999   69007800  
2022-12-28  126.040001   85438400  
2022-12-29  129.610001   75703700  
2022-12-30  129.929993   76960600  
2023-01-03  125.070000  112117500  
2023-01-04  126.360001   89113600  
2023-01-05  125.019997   80962700  
2023-01-06  129.619995   87686600  
2023-01-09  130.149994   70790800  
2023-01-10  130.729996   63896200  
2023-01-11  1

In [266]:
def getMonthlyStockData(symbol, day, interval):
    print("Getting stock data for stock $"+symbol)
    df = yf.download(symbol, start=day, period = interval, group_by='ticker')
    return df

getMonthlyStockData('AAPL', "2022-11-21", '1mo')

Getting stock data for stock $AAPL
[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-11-21,150.160004,150.369995,147.720001,148.009995,148.009995,58724100
2022-11-22,148.130005,150.419998,146.929993,150.179993,150.179993,51804100
2022-11-23,149.449997,151.830002,149.339996,151.070007,151.070007,58301400
2022-11-25,148.309998,148.880005,147.119995,148.110001,148.110001,35195900
2022-11-28,145.139999,146.639999,143.380005,144.220001,144.220001,69246000
2022-11-29,144.289993,144.809998,140.350006,141.169998,141.169998,83763800
2022-11-30,141.399994,148.720001,140.550003,148.029999,148.029999,111380900
2022-12-01,148.210007,149.130005,146.610001,148.309998,148.309998,71250400
2022-12-02,145.960007,148.000000,145.649994,147.809998,147.809998,65447400


# Gathering FinViz Data (Today's News) #

In [254]:
# Parameters 
n = 3 #the # of article headlines displayed per ticker
tickers = ['AAPL']

In [255]:
from finvizfinance.news import News
fnews = News()
all_news = fnews.get_news()
all_news

{'news':        Date                                              Title  \
 0   01:53PM  Stocks Take Leg Higher on Fed Waller’s Comment...   
 1   01:51PM  MarketWatch Live: U.S. stocks move higher but ...   
 2   01:22PM   Federal Reserve Probes Goldman Consumer Business   
 3   01:10PM  Yellen warns of 'global financial crisis' if U...   
 4   01:07PM  : Baker Hughes data show a decline in active U...   
 ..      ...                                                ...   
 85  04:51AM  IMF's Georgieva and ECB's Lagarde discuss the ...   
 86  04:25AM  Swiss Minister Signals Support Using Russian A...   
 87  04:22AM  Global equity funds post second weekly inflows...   
 88  04:00AM    Why the January Stock Market Bounce Is Illusory   
 89  03:55AM  European shares gain on China's reopening opti...   
 
                  Source                                               Link  
 0     www.bloomberg.com  https://www.bloomberg.com/news/articles/2023-0...  
 1   www.marketwatch.com  http

In [256]:
# Get Data
finwiz_url = 'https://finviz.com/quote.ashx?t='
news_tables = {}

for ticker in tickers:
    url = finwiz_url + ticker
    print("current url is: " +url)
    header = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36."}
    req = Request(url=url,headers=header) 
    resp = urlopen(req)    
    html = BeautifulSoup(resp, features="lxml")
    news_table = html.find(id='news-table')
    news_tables[ticker] = news_table

try:
    for ticker in tickers:
        df = news_tables[ticker]
        df_tr = df.findAll('tr')
    
        print ('\n')
        print ('Recent News Headlines for {}: '.format(ticker))
        
        for i, table_row in enumerate(df_tr):
            a_text = table_row.a.text
            td_text = table_row.td.text
            td_text = td_text.strip()
            print(a_text,'(',td_text,')')
            if i == n-1:
                break
except KeyError:
    pass

current url is: https://finviz.com/quote.ashx?t=AAPL


Recent News Headlines for AAPL: 
Analyst Report: Apple Inc. ( Jan-20-23 12:41PM )
Heres Why You Should Consider Investing in Apple (AAPL) ( 12:32PM )
Apple wins appeal to keep $308 million U.S. patent verdict at bay ( 12:08PM )


In [257]:
# Iterate through the news
parsed_news = []
for file_name, news_table in news_tables.items():
    for x in news_table.findAll('tr'):
        print(x.get_text())
        text = x.get_text() 
        date_scrape = x.td.text.split()

        if len(date_scrape) == 1:
            time = date_scrape[0]
            
        else:
            date = date_scrape[0]
            time = date_scrape[1]

        ticker = file_name.split('_')[0]
        
        parsed_news.append([ticker, date, time, text ])

Jan-20-23 12:41PMAnalyst Report: Apple Inc. Morningstar Research
12:32PMHeres Why You Should Consider Investing in Apple (AAPL) Insider Monkey
12:08PMApple wins appeal to keep $308 million U.S. patent verdict at bay Reuters
10:51AMThis Startup Uses AI To Leave Apple's AirTag In The Dust Benzinga
10:00AM1 Huge Reason Stocks Could Rally in 2023 Motley Fool

08:03AM
Loading…

08:03AM2 Top Warren Buffett Stocks to Buy in 2023 and Hold Forever Motley Fool
07:41AMApple appeals investigation by UK competition watchdog Reuters
07:38AMApple appeals investigation by UK competition watchdog Reuters
06:27AMApple Stock: Bull vs. Bear Motley Fool
06:03AMThe Global Smartphone Market Just Fell Off a Cliff. Apple Beat the Odds. Motley Fool
05:55AM3 Stocks Warren Buffett Is Almost Certainly Buying in 2023 Motley Fool
05:06AM89% of Warren Buffett's Secret Portfolio Is Invested in Just 5 Stocks Motley Fool
Jan-19-23 09:45AMApple Stock Is Down 25% From Its High. Time to Buy? Motley Fool
08:45AMNew to Inves

# Gathering Data From AlphaAdvantage for Historical News #

In [258]:
from decouple import config
import requests
import urllib.parse
import json
AAapikey = config('AAKey')

In [259]:
#Function to convert user provided date to date required by AlphaAdvantage
def toAADate(oldDate):
    newDate = str(oldDate) + 'T0001'
    return newDate

In [260]:
# Get data from AlphaAdvantage for one ticker for a particular day
def getHistoricNewsData(ticker, startDate = None, endDate = None):
    url = 'https://www.alphavantage.co/query?'
    if startDate and endDate:
        startDate = toAADate(startDate)
        endDate = toAADate(endDate)
        Myparams = {'function': 'NEWS_SENTIMENT', 'tickers': ticker, 'time_from': startDate, 'time_to': endDate, 'sort': 'LATEST','limit': 100, 'apikey': AAapikey}
    else:
        Myparams = {'function': 'NEWS_SENTIMENT', 'tickers': ticker, 'sort': 'LATEST','limit': 100, 'apikey': AAapikey}
    r = requests.get(url, params = Myparams)
    data = r.json()
    #return data
    historic_news = pd.DataFrame(columns=['Date', 'Headline', 'Ticker'])
    for i in data.get("feed"):
        test_date = i.get("time_published")
        test_date = test_date[:8]
        row = [test_date, i.get("title"), ticker]
        new_df = pd.DataFrame([row],columns=['Date', 'Headline', 'Ticker'])
        historic_news = pd.concat([historic_news, new_df], axis=0, ignore_index=True)
    return historic_news

historic_news = getHistoricNewsData('AAPL', '2023012', '20230117')
print(historic_news)

        Date                                           Headline Ticker
0   20230120  Microsoft and Alphabet Deepen the White-Collar...   AAPL
1   20230120  Amazon's Alexa Smart Speaker Blazed A Trail, T...   AAPL
2   20230120  Five Things We Don't Still Don't Know About Ne...   AAPL
3   20230120  Netflix  ( NFLX )  Q4 Earnings Miss, Revenues ...   AAPL
4   20230120  Major tech layoffs from Alphabet and Microsoft...   AAPL
5   20230120  This Startup Uses AI To Leave Apple's AirTag I...   AAPL
6   20230120  Could Alphabet Cuts Spell a Potentially Tough ...   AAPL
7   20230120           1 Huge Reason Stocks Could Rally in 2023   AAPL
8   20230120  This Solar Company's 5-Year Return Outshines T...   AAPL
9   20230120  EXCLUSIVE: Poll Reveals Top Picks For A $25,00...   AAPL
10  20230120  LG ELECTRONICS USA DEBUTS ITS NEW CINEBEAM SMA...   AAPL
11  20230120                 Futures Fall Ahead Of Housing Data   AAPL
12  20230120  DraftKings, FanDuel And These Streaming Apps C...   AAPL
13  20

# Sentiment Analysis of News data #

In [261]:
def SentimentAnalysisNewsData(parsedNews):
    nltk.download('vader_lexicon')
    # Sentiment Analysis
    analyzer = SentimentIntensityAnalyzer()

    columns = ['Ticker', 'Date', 'Time', 'Headline']
    news = pd.DataFrame(parsedNews, columns=columns)
    scores = news['Headline'].apply(analyzer.polarity_scores).tolist()

    df_scores = pd.DataFrame(scores)
    news = news.join(df_scores, rsuffix='_right')
    news['Date'] = pd.to_datetime(news.Date).dt.date

    unique_ticker = news['Ticker'].unique().tolist()
    news_dict = {name: news.loc[news['Ticker'] == name] for name in unique_ticker}

    values = []
    for ticker in tickers: 
        dataframe = news_dict[ticker]
        dataframe = dataframe.set_index('Ticker')
        dataframe = dataframe.drop(columns = ['Headline'])
        print ('\n')
        print (dataframe.head())
        
        mean = round(dataframe['compound'].mean(), 2)
        values.append(mean)
        
    df = pd.DataFrame(list(zip(tickers, values)), columns =['Ticker', 'Mean Sentiment']) 
    df = df.set_index('Ticker')
    df = df.sort_values('Mean Sentiment', ascending=False)
    print ('\n')
    print (df)
    return df

In [262]:
HistoricSentiment = SentimentAnalysisNewsData(historic_news)
TodaysSentiment = SentimentAnalysisNewsData(parsed_news)



              Date  Time    neg    neu    pos  compound
Ticker                                                 
AAPL    2023-01-20   NaN  0.318  0.682  0.000   -0.4215
AAPL    2023-01-20   NaN  0.000  0.748  0.252    0.4019
AAPL    2023-01-20   NaN  0.000  1.000  0.000    0.0000
AAPL    2023-01-20   NaN  0.114  0.643  0.243    0.4215
AAPL    2023-01-20   NaN  0.000  0.870  0.130    0.3400


        Mean Sentiment
Ticker                
AAPL              0.09


              Date     Time    neg    neu    pos  compound
Ticker                                                    
AAPL    2023-01-20  12:41PM  0.000  1.000  0.000    0.0000
AAPL    2023-01-20  12:32PM  0.000  1.000  0.000    0.0000
AAPL    2023-01-20  12:08PM  0.000  0.675  0.325    0.6486
AAPL    2023-01-20  10:51AM  0.098  0.902  0.000   -0.0516
AAPL    2023-01-20  10:00AM  0.220  0.606  0.174   -0.1531


        Mean Sentiment
Ticker                
AAPL             -0.04


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Ishaan\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Ishaan\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


# Creating Dataset #

In [263]:
for i in tickers:
    

IndentationError: expected an indented block after 'for' statement on line 1 (1525691964.py, line 2)